# Streaming Twitter Activity With A Bot

## Introduction

Oftentimes when working with Twitter data, it's of interest to simply listen to all activity in a given time. Luckily, as Twitter is a data company, they make this fairly easy with an Application Programming Interface (API). Through their API, we can automatically capture tweets as they happen for free (with limitations such as how many tweets you can capture, and how far back in time you can grab data from). Of course, if you're willing to pay, you can capture _significantly_ more data from Twitter. But for us, all we need is a free account to get going. 

## Using This Streamer
Unfortunately, in its current state, you cannot use this streamer to capture twitter data. You need to create credentials to pass into the API. In order to do that, go to this [link](https://developer.twitter.com/en/apply-for-access) create an account and follow the instructions. Once you get your API keys, you can fill in the `YOUR CREDENTIALS HERE` section of the code below. It is however, not required for this workshop as we've already captured the data for you. 

## Data We Captured

We set up our Twitter bot to capture all tweets from February 25th to March 4th that appeared from within a box around the approximate GPS coordinates of Alberta. Every time someone made a public tweet, we captured and saved this tweet to a text filed called `alberta_tweets_feb_25_start.txt`. We captured certain information about the user who tweeted, any GPS coordinates of the location of the tweet if the user had provided them, as well as some additional meta data. Using the data we captured using this streamer, we will guide you through some text mining and analysis using Python. 

In [ ]:
import json
import pandas as pd
import time

try : 
    import tweepy
    from tweepy import Stream
    from tweepy import OAuthHandler
    from tweepy.streaming import StreamListener
except ImportError:
    !pip install tweepy --user
    from importlib import reload
    import site
    reload(site)
    import tweepy
    from tweepy import Stream
    from tweepy import OAuthHandler
    from tweepy.streaming import StreamListener

auth = tweepy.OAuthHandler('YOUR CREDENTIALS HERE', 
                           'YOUR CREDENTIALS HERE')
auth.set_access_token('YOUR CREDENTIALS HERE', 
                      'YOUR CREDENTIALS HERE')

# Waiting on rate limit is important so Twitter doesn't deactivate our free bot. 
api = tweepy.API(auth, wait_on_rate_limit=True)


#This is a basic listener that just prints received tweets to stdout.

class StdOutListener(StreamListener):
    def on_status(self, status):
        try:
            jsonData = status._json
            tweetID = jsonData.get("id_str")
            tweetData = api.get_status(tweetID)

            #check if tweet is valid (not a retweet)
            if ( (hasattr(tweetData, 'retweeted_status')) ):
                pass
            else:
                try:
                     d = dict(coordinates= jsonData['coordinates'],
                            text=jsonData['text'],
                            created_at=jsonData['created_at'],
                            user_mentions= jsonData['entities']['user_mentions'],
                            hashtags= jsonData['entities']['hashtags'],
                            lang = jsonData['lang'],
                            name=jsonData['user']['name'],
                            screen_name=jsonData['user']['screen_name'],
                            user_location=jsonData['user']['location'],
                            extended_tweet=jsonData['extended_tweet']['full_text'],
                            tweet_id = jsonData['id_str'],
                            reply_to = jsonData['in_reply_to_status_id'] )

                # if tweet is too short, extended_tweet is not a property apparently. 
                except KeyError:
                    try:
                        d = dict(coordinates= jsonData['coordinates'],
                                text=jsonData['text'],
                                created_at=jsonData['created_at'],
                                user_mentions= jsonData['entities']['user_mentions'],
                                hashtags= jsonData['entities']['hashtags'],
                                lang = jsonData['lang'],
                                name=jsonData['user']['name'],
                                screen_name=jsonData['user']['screen_name'],
                                user_location=jsonData['user']['location'],
                                extended_tweet=jsonData['text'],
                                tweet_id = jsonData['id_str'],
                                reply_to = jsonData['in_reply_to_status_id'] )
                    except:
                        d = dict(coordinates= None,
                                text=None,
                                created_at= None ,
                                user_mentions= None,
                                hashtags= None,
                                lang = None,
                                name=None,
                                screen_name=None,
                                user_location=None,
                                extended_tweet=None,
                                tweet_id = None,
                                reply_to = None )
                        pass

                
                   
                print('='*60)
                print("saving new tweet, text:")
                print(d['extended_tweet'])
                print()
                with open('alberta_tweets_feb_25_start.txt','a') as f:
                    f.write(str(d))
                    f.write('\n')
                    
                

        except (tweepy.error.RateLimitError):
            print("rate limiting?, waiting for one minute")
            time.sleep(60)

        except Exception as e:
            print("something went wrong")
            print(e)
            pass


    def on_error(self, status):
        #error number 503, servers down
        #print('Error #:', status)
        pass

In [ ]:
### Test if the api is working
user = api.me()
print (user.name)

In [ ]:
## Print tweets on a screen and save into a file:

calgary = [-114.2445060672,50.8842538358,-113.8726833982,51.1844055291]
alberta = [-121.000000, 48.000000,-109.000000, 61.000000]
twitterStream = Stream(auth, StdOutListener(), tweet_mode='extended')
twitterStream.filter(languages=["en"], locations=alberta)